In [1]:
!git clone https://github.com/ddevaul/ancient-greek-pottery-restoration

Cloning into 'PotteryRestoration'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 48 (delta 15), reused 44 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (48/48), 11.77 MiB | 13.44 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [2]:
!cd PotteryRestoration
!ls
!rm -rf PotterRestoration/dataset/train/full/.ipynb_checkpoints

drive  PotteryRestoration  sample_data


In [3]:
from PotteryRestoration.dataset.VaseDataset import VaseDataset
from torchvision import transforms
from torch.utils.data import DataLoader

# image pipeline code

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
!rm -rf ./PotteryRestoration/dataset/train/full

In [41]:
import zipfile
main_path = "drive/MyDrive/Greek_Pottery_In_Painting/"
specific_data = "November_21_Images/"
full_path = main_path + specific_data + "images.zip"
BASE_DIR = "./images"
with zipfile.ZipFile(full_path,"r") as zip:
    zip.extractall(BASE_DIR)

In [42]:
import os
import random
path_to_sub_dir = BASE_DIR + "/" + os.listdir(BASE_DIR)[0]
image_names = []
for img_name in os.listdir(path_to_sub_dir):
  # full_image_path = path_to_sub_dir + "/" + img_name
  image_names.append(img_name)

random.shuffle(image_names)
print(image_names[0])

CVA.GB05.194-BLACK-FIGURE.jpg


In [43]:
num_images = len(image_names)
tenth_of_images = num_images // 10
test_images = image_names[:tenth_of_images]
val_images = image_names[tenth_of_images: 2 * tenth_of_images]
train_images = image_names[2*tenth_of_images:]

print(len(test_images), len(val_images), len(train_images), len(test_images) + len(train_images) + len(val_images))

671 671 5372 6714


In [47]:
import shutil
os.mkdir("./PotteryRestoration/dataset/train/full")
image_train_base_path = "./PotteryRestoration/dataset/train/full"
for im_name in train_images:
  shutil.move(src=f"{path_to_sub_dir}/{im_name}", dst=f"{image_train_base_path}/{im_name}")

# need to do this for val and test ?

In [48]:
import csv
data_dict = {} # img_name : caption
new_image_prefix = "BF_image"
with open(f"{main_path + specific_data}aggregrate_data.csv", "r") as f:
  lines = f.readlines()
  for line in lines:
    row = line.split(",")
    img_name = row[0].split("/")[-1]
    # new_image_name = f"{new_image_prefix}{num_images}.jpg"
    # os.rename(f"{image_train_base_path}/{img_name}", f"{image_train_base_path}/{new_image_name}")
    caption = f"{row[2]}{row[3]}"
    data_dict[img_name] = caption

# train
train_data = []
num_train_images = 0
for filename in os.listdir(image_train_base_path):
  if filename == ".ipynb_checkpoints":
    continue
  print(filename)
  new_image_name = f"{new_image_prefix}{num_train_images}.jpg"
  os.rename(f"{image_train_base_path}/{filename}", f"{image_train_base_path}/{new_image_name}")
  new_obj = {"filename": new_image_name, "caption": data_dict[filename]}
  train_data.append(new_obj)
  num_train_images += 1

with open("./PotteryRestoration/dataset/train/captions.csv", "w") as csvfile:
  fields = ["filename", "caption"]
  csvfilewriter = csv.DictWriter(csvfile, fieldnames=fields)
  csvfilewriter.writeheader()
  csvfilewriter.writerows(train_data)


Streaming output truncated to the last 5000 lines.
3469-BLACK-FIGURE.jpg
CVA.GB15.673-BLACK-FIGURE.jpg
CVA.POL02.085-BLACK-FIGURE.jpg
CVA.GER35.1699-BLACK-FIGURE.jpg
CVA.SWZ04.187-BLACK-FIGURE.jpg
11957-BLACK-FIGURE.jpg
CVA.155-BLACK-FIGURE.jpg
CVA.POL02.094-BLACK-FIGURE.jpg
CVA.GR01.021-BLACK-FIGURE.jpg
390040-BLACK-FIGURE.jpg
46901-BLACK-FIGURE.jpg
390089-BLACK-FIGURE.jpg
310032.A-BLACK-FIGURE.jpg
CVA.GER30.1440-BLACK-FIGURE.jpg
CVA.IT70.3186-BLACK-FIGURE.jpg
CVA.GR12.817-BLACK-FIGURE.jpg
320184-BLACK-FIGURE.jpg
306664-BLACK-FIGURE.jpg
CVA.USA02.063-BLACK-FIGURE.jpg
9855-BLACK-FIGURE.jpg
CVA.GER32.1542-BLACK-FIGURE.jpg
CVA.IT25.1151-BLACK-FIGURE.jpg
CVA.GER102.5127-BLACK-FIGURE.jpg
301892-BLACK-FIGURE.jpg
CVA.USA18.846-BLACK-FIGURE.jpg
CVA.GR12.812-BLACK-FIGURE.jpg
CVA.USA23.1129-BLACK-FIGURE.jpg
361507-BLACK-FIGURE.jpg
6229-BLACK-FIGURE.jpg
CVA.F30.1342-BLACK-FIGURE.jpg
CVA.GER48.2326-BLACK-FIGURE.jpg
9032536-BLACK-FIGURE.jpg
CVA.POL01.008-BLACK-FIGURE.jpg
310322-BLACK-FIGURE.jpg
CV

In [49]:
# val
import shutil
os.mkdir("./PotteryRestoration/dataset/val/full")
image_val_base_path = "./PotteryRestoration/dataset/val/full"
for im_name in val_images:
  shutil.move(src=f"{path_to_sub_dir}/{im_name}", dst=f"{image_val_base_path}/{im_name}")


val_data = []
num_val_images = 0
for filename in os.listdir(image_val_base_path):
  if filename == ".ipynb_checkpoints":
    continue
  new_image_name = f"{new_image_prefix}{num_val_images}.jpg"
  os.rename(f"{image_val_base_path}/{filename}", f"{image_val_base_path}/{new_image_name}")
  new_obj = {"filename": new_image_name, "caption": data_dict[filename]}
  val_data.append(new_obj)
  num_train_images += 1

with open("./PotteryRestoration/dataset/val/captions.csv", "w") as csvfile:
  fields = ["filename", "caption"]
  csvfilewriter = csv.DictWriter(csvfile, fieldnames=fields)
  csvfilewriter.writeheader()
  csvfilewriter.writerows(val_data)

In [ ]:
# Not sure if we need transformations?
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(), # Convert images to PyTorch tensors
])

# Make datasets
train_dataset = VaseDataset(root_dir="PotteryRestoration/dataset/train", captions_file="captions.csv", transform=transform)

val_dataset = VaseDataset(root_dir="PotteryRestoration/dataset/val", captions_file="captions.csv", transform=transform)


# Make dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [53]:
!python ./PotteryRestoration/dataset/mask_images.py

Processing images in ./PotteryRestoration/dataset/train: 100% 5372/5372 [1:00:12<00:00,  1.49it/s]
Saved mappings to ./PotteryRestoration/dataset/train/mappings.csv
Processing images in ./PotteryRestoration/dataset/val: 100% 1/1 [00:00<00:00,  1.69it/s]
Saved mappings to ./PotteryRestoration/dataset/val/mappings.csv


In [70]:
!mkdir /content/drive/MyDrive/Greek_Pottery_In_Painting/nov_23_image_and_mask/

In [ ]:
!zip -r /content/drive/MyDrive/Greek_Pottery_In_Painting/nov_23_image_and_mask/train.zip ./PotteryRestoration/dataset/train

Streaming output truncated to the last 5000 lines.
  adding: PotteryRestoration/dataset/train/masked/BF_image4616_masked_9.png (deflated 2%)
  adding: PotteryRestoration/dataset/train/masked/BF_image4390_masked_7.png (deflated 1%)
  adding: PotteryRestoration/dataset/train/masked/BF_image2968_masked_0.png (deflated 1%)
  adding: PotteryRestoration/dataset/train/masked/BF_image2846_masked_1.png (deflated 1%)
  adding: PotteryRestoration/dataset/train/masked/BF_image4110_masked_0.png (deflated 1%)
  adding: PotteryRestoration/dataset/train/masked/BF_image4590_masked_5.png (deflated 1%)
  adding: PotteryRestoration/dataset/train/masked/BF_image1698_masked_8.png (deflated 2%)
  adding: PotteryRestoration/dataset/train/masked/BF_image3057_masked_3.png (deflated 2%)
  adding: PotteryRestoration/dataset/train/masked/BF_image1610_masked_1.png (deflated 1%)
  adding: PotteryRestoration/dataset/train/masked/BF_image2350_masked_5.png (deflated 1%)
  adding: PotteryRestoration/dataset/train/masked

In [66]:
!zip -r ./PotteryRestoration/dataset/train . -i /content/drive/MyDrive/Greek_Pottery_In_Painting/nov_23_image_and_mask/train

	zip warning: zip file empty


In [ ]:
from diffusers import StableDiffusionInpaintPipeline
from accelerate import Accelerator
import torch
from torch.utils.data import DataLoader
from transformers import AdamW

# Load pipeline
pipe = StableDiffusionInpaintPipeline.from_pretrained("stabilityai/stable-diffusion-2-inpainting")
device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"

# Freeze VAE parameters
for param in pipe.vae.parameters():
    param.requires_grad = False

# Fine-tune only the U-Net and text encoder
for param in pipe.unet.parameters():
    param.requires_grad = True
for param in pipe.text_encoder.parameters():
    param.requires_grad = True

pipe.to(device)

# Optimizer
optimizer = AdamW(
    [{"params": pipe.unet.parameters()}, {"params": pipe.text_encoder.parameters()}],
    lr=5e-5
)

# DataLoader placeholder (replace `train_dataloader` with your actual DataLoader)
# train_dataloader = DataLoader(...)

# Use Accelerator for distributed training
accelerator = Accelerator()
pipe, optimizer, train_dataloader = accelerator.prepare(pipe, optimizer, train_dataloader)

NUM_EPOCHS = 2

# Training loop
for epoch in range(NUM_EPOCHS):
    pipe.unet.train()
    pipe.text_encoder.train()

    for batch in train_dataloader:
        # Get inputs
        masked_images = batch["masked_images"].to(device)
        full_images = batch["full_images"].to(device)
        masks = batch["masks"].to(device)  # Binary masks
        prompts = batch["text"]

        # Tokenize text prompts
        tokenized_prompts = pipe.tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(device)
        text_embeddings = pipe.text_encoder(**tokenized_prompts).last_hidden_state

        # Encode masked images into latent space
        latents = pipe.vae.encode(masked_images).latent_dist.sample()
        latents = latents * pipe.vae.config.scaling_factor

        # Assert latent dimensions
        assert latents.shape[1] == 4, f"Latent channels should be 4, got {latents.shape[1]}"
        assert latents.shape[2] % 8 == 0 and latents.shape[3] % 8 == 0, \
            "Latent dimensions should be divisible by 8 for the UNet"

        # Add noise to the latents
        batch_size = latents.size(0)
        timesteps = torch.randint(0, pipe.scheduler.num_train_timesteps, (batch_size,), device=device).long()
        noise = torch.randn_like(latents)
        noisy_latents = pipe.scheduler.add_noise(latents, noise, timesteps)

        # Resize mask to match latent spatial dimensions
        latent_masks = torch.nn.functional.interpolate(masks, size=noisy_latents.shape[-2:])  # Resize mask
        latent_masks = latent_masks[:, None, :, :]  # Ensure shape is (B, 1, H, W)

        # Fix latent_masks shape by squeezing the extra singleton dimension
        if latent_masks.ndim == 5:  # Check if there's an extra dimension
            latent_masks = latent_masks.squeeze(2)  # Remove the extra dimension
        # Assert mask shape matches expected dimensions
        print(f"Noisy latents shape: {noisy_latents.shape}")
        print(f"Latent masks shape: {latent_masks.shape}")

        assert latent_masks.ndim == 4, f"Mask should have 4 dimensions, got {latent_masks.ndim}"
        assert latent_masks.shape[1] == 1, f"Mask must have 1 channel, got {latent_masks.shape[1]}"
        assert latent_masks.shape[2:] == noisy_latents.shape[2:], \
            f"Mask spatial dimensions {latent_masks.shape[2:]} must match latents {noisy_latents.shape[2:]}"

        # Generate spatial encodings
        batch_size, _, height, width = noisy_latents.shape
        x = torch.linspace(-1, 1, steps=width, device=device).view(1, 1, 1, -1).expand(batch_size, 1, height, width)
        y = torch.linspace(-1, 1, steps=height, device=device).view(1, 1, -1, 1).expand(batch_size, 1, height, width)
        spatial_encodings = torch.cat([x, y], dim=1)  # Shape: (B, 2, H, W)

        # Concatenate noisy latents, mask, and spatial encodings
        unet_input = torch.cat([noisy_latents, latent_masks, spatial_encodings], dim=1)

        # Add extra dummy channels (if required)
        extra_channels = torch.zeros(unet_input.shape[0], 2, unet_input.shape[2], unet_input.shape[3], device=device)
        unet_input = torch.cat([unet_input, extra_channels], dim=1)

        # Assert the input shape
        assert unet_input.shape[1] == 9, f"UNet input must have 9 channels, got {unet_input.shape[1]}"

        # Forward pass through UNet
        unet_output = pipe.unet(
            sample=unet_input,
            timestep=timesteps,
            encoder_hidden_states=text_embeddings
        ).sample

        # Assert UNet output shape matches latent input
        assert unet_output.shape == latents.shape, \
            f"UNet output shape mismatch: {unet_output.shape} != {latents.shape}"

        # Decode the output latents back to image space
        reconstructed_images = pipe.vae.decode(unet_output / pipe.vae.config.scaling_factor).sample

        # Assert decoded images match the size of full images
        assert reconstructed_images.shape == full_images.shape, \
            f"Decoded images shape mismatch: {reconstructed_images.shape} != {full_images.shape}"

        # Compute pixel-wise loss
        loss = torch.nn.functional.mse_loss(reconstructed_images, full_images)

        # Backpropagation
        accelerator.backward(loss)
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch + 1}/{NUM_EPOCHS} completed. Loss: {loss.item():.4f}")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Noisy latents shape: torch.Size([1, 4, 32, 32])
Latent masks shape: torch.Size([1, 1, 32, 32])
Noisy latents shape: torch.Size([1, 4, 32, 32])
Latent masks shape: torch.Size([1, 1, 32, 32])
Noisy latents shape: torch.Size([1, 4, 32, 32])
Latent masks shape: torch.Size([1, 1, 32, 32])
Noisy latents shape: torch.Size([1, 4, 32, 32])
Latent masks shape: torch.Size([1, 1, 32, 32])
Noisy latents shape: torch.Size([1, 4, 32, 32])
Latent masks shape: torch.Size([1, 1, 32, 32])
Noisy latents shape: torch.Size([1, 4, 32, 32])
Latent masks shape: torch.Size([1, 1, 32, 32])
Noisy latents shape: torch.Size([1, 4, 32, 32])
Latent masks shape: torch.Size([1, 1, 32, 32])
Noisy latents shape: torch.Size([1, 4, 32, 32])
Latent masks shape: torch.Size([1, 1, 32, 32])
Noisy latents shape: torch.Size([1, 4, 32, 32])
Latent masks shape: torch.Size([1, 1, 32, 32])
Noisy latents shape: torch.Size([1, 4, 32, 32])
Latent masks shape: torch.Size([1, 1, 32, 32])
Noisy latents shape: torch.Size([1, 4, 32, 32])
La

In [ ]:
!rm -rf `find -type d -name .ipynb_checkpoints`